<a href="https://colab.research.google.com/github/malloyca/CSC581B/blob/main/Final%20Project/linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC581B - Introduction to Deep Learning for Image Classification
# Final Project

In [1]:
# imports
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

First we need to load the training and test data.

In [55]:
# Load the training data (CIFAR10 to start)
training_data = datasets.CIFAR10(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

# Load the test data
test_data = datasets.CIFAR10(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

'''training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)'''

Files already downloaded and verified
Files already downloaded and verified


'training_data = datasets.FashionMNIST(\n    root = "data",\n    train = True,\n    download = True,\n    transform = ToTensor()\n)\n\ntest_data = datasets.FashionMNIST(\n    root = "data",\n    train = False,\n    download = True,\n    transform = ToTensor()\n)'

In [66]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [67]:
for X, y in test_dataloader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 3, 32, 32])
Shape of y:  torch.Size([64]) torch.int64


## Building a first model

In [68]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [69]:
# Define the model
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(32*32*3, 2048),
        #nn.ReLU(),
        nn.Linear(2048, 2048),
        #nn.ReLU(),
        nn.Linear(2048, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNet().to(device)
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=2048, bias=True)
    (1): Linear(in_features=2048, out_features=2048, bias=True)
    (2): Linear(in_features=2048, out_features=10, bias=True)
  )
)


In [70]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [71]:
# Define training loop function
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [72]:
# Define the test function
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [73]:
n_epochs = 5

In [74]:
for t in range(n_epochs):
  print(f"Epoch {t+1}\n----------------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done.")

Epoch 1
----------------------------------------
loss: 2.319583 [    0/50000]
loss: 2.302674 [ 6400/50000]
loss: 2.248554 [12800/50000]
loss: 2.262769 [19200/50000]
loss: 2.213516 [25600/50000]
loss: 2.208179 [32000/50000]
loss: 2.229230 [38400/50000]
loss: 2.172769 [44800/50000]
Test Error: 
 Accuracy: 23.9%, Avg loss: 2.165262 

Epoch 2
----------------------------------------
loss: 2.210411 [    0/50000]
loss: 2.188001 [ 6400/50000]
loss: 2.073062 [12800/50000]
loss: 2.177170 [19200/50000]
loss: 2.095382 [25600/50000]
loss: 2.096824 [32000/50000]
loss: 2.177542 [38400/50000]
loss: 2.067110 [44800/50000]
Test Error: 
 Accuracy: 27.3%, Avg loss: 2.072367 

Epoch 3
----------------------------------------
loss: 2.134713 [    0/50000]
loss: 2.109297 [ 6400/50000]
loss: 1.949307 [12800/50000]
loss: 2.113304 [19200/50000]
loss: 2.032635 [25600/50000]
loss: 2.027601 [32000/50000]
loss: 2.148042 [38400/50000]
loss: 2.002616 [44800/50000]
Test Error: 
 Accuracy: 29.7%, Avg loss: 2.012936 

E

In [75]:
# Save the model
torch.save(model.state_dict(), "linear_model.pth")